In [1]:
import re
import sys
import numpy as np
import operator
import random
import nltk
import spacy
from sklearn.linear_model import LogisticRegression
from fetch_gutenberg import identity_preprocess, clean_text, tokenize, preprocess, process_file, download_corpus
from helpers import read_wiki_txt
from w2v_sentiment_analysis import strip_punctuation, preprocess, tokenize, preproc_tok, read_samples, create_corpus
from nltk.tokenize import word_tokenize
from helpers import run_cmd
from util import format_arc, EPS, CHARS
import re
import contractions
from mkfstinput import makefstinput_
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, KeyedVectors

#### Step 1

In [2]:
#step1 results
gut_path = "../data/gutenberg.txt"
with open(gut_path, 'r') as file:
    gutenberg_txt = file.read()
print(gutenberg_txt,type(gutenberg_txt),np.shape(gutenberg_txt))
preprocessed_sentences = process_file(gutenberg_txt,preprocess=preprocess)
preprocessed_temp = []
for sentence in preprocessed_sentences:
    preprocessed_temp.append([w for w in sentence.split(" ") if len(w) > 0])
preprocessed_words = [item for sublist in preprocessed_temp for item in sublist]
print(np.shape(preprocessed_words),type(preprocessed_words[0]),preprocessed_words[:10])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(2134975,) <class 'str'> ['emma', 'by', 'jane', 'austen', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse']


#### Step 2

In [3]:
#step2
def word_count(txt):
    counts = dict()
    words = txt.split()
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1 
    out_dict= dict(sorted(((k, v) for k, v in counts.items() if v >= 5), key=operator.itemgetter(1),reverse=True))
    keys = np.array([key for key in out_dict])
    values = np.array([str(out_dict[key]) for key in out_dict])
    np.savetxt("../vocab/words.vocab.txt",np.transpose([keys,values]),delimiter=' ',fmt='%s')
    return out_dict, keys, values
out_dict, _, _= word_count(gutenberg_txt)

#### Step 3

In [4]:
#step3
def write_syms(txt):
    #write alphabet file
    alphabet = np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', 'ﬂ'])
    f = open("../vocab/chars.syms","w")
    f.write("<eps>" + " " + str(0) + '\n')
    num = 97
    for character in alphabet:
        f.write(character + " " + str(num) + '\n')
        num += 1
    f.close()
    #write tokens file
    _,tokens,_ = word_count(txt)
    f = open("../vocab/words.syms","w+")
    f.write("<eps>"+ " " + str(0) + '\n')
    num=1
    for token in tokens:
        f.write(token + " " + str(num) + '\n')
        num += 1
    f.close()    
write_syms(gutenberg_txt)

#### Step 4

In [5]:
def create_L(charset, filename):
    with open(filename, "w") as f:
        for c in charset:
            print(format_arc(0, 0, c, c, weight=0), file=f)   # 1: Every character to itself with weight 0 (no edit)
            print(format_arc(0, 0, c, EPS, weight=1), file=f) # 2: Every character to EPS with weight 1 (deletion)
            print(format_arc(0, 0, EPS, c, weight=1), file=f) # 3: EPS to every other character with weight 1 (insertion)
            
            for other_c in charset:                           # 4: Every character to every other character with weight 1
                if c == other_c:
                    continue # If the characters are the same the weight is 0
                print(format_arc(0, 0, c, other_c, weight=1), file=f)
        print(0, file=f)  # Accepting state
create_L(CHARS, "../fsts/L.fst")
# create_L_transducer(CHARS[:3], "../fsts/L.fst")



In [6]:
#step4 - shell script
! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/L.fst > ../fsts/L.binfst
# ! fstdraw --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms -portrait ../fsts/L.binfst | dot -Tpng >../documentation/L.png

#### Step 5

In [7]:
def create_V(dictionary, filename,num_words=-1):
    s = 1
    if num_words == -1:
        # Accept state number should be large enough so it doesn't mess with other states
        accept_state = sum([len(word) for word in dictionary]) + 1 

        with open(filename, "w") as f:
            for word in dictionary:
                for i, c in enumerate(word):
                    if i == 0:
                        # First letter of each word.
                        # Output whole word at first letter (we will then output <eps> for subsequent letters)
                        print(format_arc(0, s, c, word, weight=0), file=f) 
                    else:
                        # Rest of letters
                        print(format_arc(s, s + 1, c, EPS, weight=0), file=f)
                        s += 1

                    if i == len(word) - 1:
                        # End of word. Connect with accepting state through an epsilon transition
                        print(format_arc(s, accept_state, EPS, EPS, weight=0), file=f)
                s += 1
            print(accept_state, file=f)
    else:
        accept_state = num_words
        current_word = 0
        with open(filename, "w") as f:
            for word in dictionary:
                if current_word >= num_words:
                    break
                current_word = current_word + 1
                for i, c in enumerate(word):
                    if i == 0:
                        # First letter of each word.
                        # Output whole word at first letter (we will then output <eps> for subsequent letters)
                        print(format_arc(0, s, c, word, weight=0), file=f) 
                    else:
                        # Rest of letters
                        print(format_arc(s, s + 1, c, EPS, weight=0), file=f)
                        s += 1

                    if i == len(word) - 1:
                        # End of word. Connect with accepting state through an epsilon transition
                        print(format_arc(s, accept_state, EPS, EPS, weight=0), file=f)
                s += 1
            print(accept_state, file=f)
create_V(out_dict, "../fsts/V_arx.fst")

In [8]:
# #step 5 - shell scripting
! fstcompile -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V_arx.fst ../fsts/V_arx.binfst
# ! fstdraw --isymbols=../vocab/chars.syms --osymbols=../vocab/words.syms -portrait ../fsts/V_arx.binfst | dot -Tpng >../documentation/V_arx.png

! fstrmepsilon ../fsts/V_arx.binfst ../fsts/V_arx.binfst
# ! fstdraw --isymbols=../vocab/chars.syms --osymbols=../vocab/words.syms -portrait ../fsts/V_arx.binfst | dot -Tpng >../documentation/V_arx_2.png
! fstdeterminize ../fsts/V_arx.binfst ../fsts/V_arx.binfst
# ! fstdraw --isymbols=../vocab/chars.syms --osymbols=../vocab/words.syms -portrait ../fsts/V_arx.binfst | dot -Tpng >../documentation/V_arx_3.png
! fstminimize ../fsts/V_arx.binfst ../fsts/V.binfst
! fstprint -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V.binfst > ../fsts/V.fst
! fstcompile -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V.fst > ../fsts/V.binfst
# ! fstdraw --isymbols=../vocab/chars.syms --osymbols=../vocab/words.syms -portrait ../fsts/V.binfst | dot -Tpng >../documentation/V.png

#### Step 6

In [9]:
#step6
! fstarcsort --sort_type=olabel ../fsts/L.binfst ../fsts/L_sorted.binfst
! fstarcsort --sort_type=ilabel ../fsts/V.binfst ../fsts/V_sorted.binfst
! fstcompose ../fsts/L_sorted.binfst ../fsts/V_sorted.binfst ../fsts/S.binfst

#### Step 7

In [10]:
#step 7
def make_input_fsts(word,filename="../fsts/S_text.fst"):
    """Create an fst that accepts a word letter by letter
    This can be composed with other FSTs, e.g. the spell
    checker to provide an "input" word
    """
    s, accept_state = 0, 500000
    with open(filename, "w") as f:
        # print(f)
        for i, c in enumerate(word):
            print(format_arc(s, s + 1, c, c, weight=0),file=f)
            s += 1
            
            if i == len(word) - 1:
                print(format_arc(s, accept_state, EPS, EPS, weight=0),file=f)

        print(accept_state,file=f)

def run_ev_cmds():
    ! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/S_text.fst > ../fsts/S_text.binfst
    ! fstarcsort --sort_type=olabel ../fsts/S_text.binfst ../fsts/S_text.binfst
    ! fstarcsort --sort_type=ilabel ../fsts/EV.binfst ../fsts/EV.binfst
    ! fstcompose ../fsts/S_text.binfst ../fsts/EV.binfst ../fsts/kati.binfst

def run_evao_cmds():
    ! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/S_text.fst > ../fsts/S_text.binfst
    ! fstarcsort --sort_type=olabel ../fsts/S_text.binfst ../fsts/S_text.binfst
    ! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/E_add_one.fst > ../fsts/E_add_one.binfst
    ! fstarcsort --sort_type=olabel ../fsts/E_add_one.binfst ../fsts/E_add_one_sorted.binfst
    ! fstcompose ../fsts/E_add_one_sorted.binfst ../fsts/V_sorted.binfst ../fsts/E_AO_V.binfst
    ! fstarcsort --sort_type=ilabel ../fsts/E_AO_V.binfst ../fsts/E_AO_V_sorted.binfst
    ! fstcompose ../fsts/S_text.binfst ../fsts/E_AO_V_sorted.binfst ../fsts/kati.binfst
    
def run_s_cmds():
    ! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/S_text.fst > ../fsts/S_text.binfst
    ! fstarcsort --sort_type=olabel ../fsts/S_text.binfst ../fsts/S_text.binfst
    ! fstarcsort --sort_type=ilabel ../fsts/S.binfst ../fsts/S.binfst
    ! fstcompose ../fsts/S_text.binfst ../fsts/S.binfst ../fsts/kati.binfst
    
def run_lvw_cmds():
    ! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/S_text.fst > ../fsts/S_text.binfst
    ! fstarcsort --sort_type=olabel ../fsts/S_text.binfst ../fsts/S_text.binfst
    ! fstarcsort --sort_type=ilabel ../fsts/LVW.binfst ../fsts/LVW.binfst
    ! fstcompose ../fsts/S_text.binfst ../fsts/LVW.binfst ../fsts/kati.binfst

def run_lvw2_cmds():
    ! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/S_text.fst > ../fsts/S_text.binfst
    ! fstarcsort --sort_type=olabel ../fsts/S_text.binfst ../fsts/S_text.binfst
    ! fstarcsort --sort_type=ilabel ../fsts/LVW_2.binfst ../fsts/LVW_2.binfst
    ! fstcompose ../fsts/S_text.binfst ../fsts/LVW_2.binfst ../fsts/kati.binfst

def run_evw2_cmds():
    ! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/S_text.fst > ../fsts/S_text.binfst
    ! fstarcsort --sort_type=olabel ../fsts/S_text.binfst ../fsts/S_text.binfst
    ! fstarcsort --sort_type=ilabel ../fsts/EVW_2.binfst ../fsts/EVW_2.binfst
    ! fstcompose ../fsts/S_text.binfst ../fsts/EVW_2.binfst ../fsts/kati.binfst

def run_evw_cmds():
    ! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/S_text.fst > ../fsts/S_text.binfst
    ! fstarcsort --sort_type=olabel ../fsts/S_text.binfst ../fsts/S_text.binfst
    ! fstarcsort --sort_type=ilabel ../fsts/EVW.binfst ../fsts/EVW.binfst
    ! fstcompose ../fsts/S_text.binfst ../fsts/EVW.binfst ../fsts/kati.binfst
        
def test_spell_checker(test_file, spellchecker,test_set_size=10):
    score=0
    corrected_results=[]
    wrong_results=[]
    correct_results=[]
    num_of_words = 0
    with open(test_file) as fin:
        for _ in range(test_set_size):
            words = fin.readline().strip().split(" ") # Get all words in a line
            correct_word = words[0][:-1] # Throw away the colon (e.g. contended: --> contended)
            wrong_words = words[1:] # The rest of the words in a line are wrong
            for wrong_word in wrong_words:
                num_of_words+=1
                # wrong_word = "mountian"
                # print("wrong word is: ", wrong_word)
                # print("correct word is: ",correct_word)
                correct_results.append(correct_word)
                wrong_results.append(wrong_word)
                make_input_fsts(wrong_word)
                if(spellchecker=="EV"):
                    # print("mpikaEV")
                    run_ev_cmds()
                if(spellchecker=="E_AO_V"):
                    # print("mpikaEV")
                    run_evao_cmds()
                if(spellchecker=="S"):
                    # print("mpikaS")
                    run_s_cmds()
                if(spellchecker=="LVW"):
                    # print("mpikaLVW")
                    run_lvw_cmds()
                if(spellchecker=="EVW"):
                    # print("mpikaEVW")
                    run_evw_cmds()
                if(spellchecker=="EVW_2"):
                    # print("mpikaEVW")
                    run_evw2_cmds()
                if(spellchecker=="LVW_2"):
                    # print("mpikaEVW")
                    run_lvw2_cmds()
                ! fstshortestpath ../fsts/kati.binfst | fstrmepsilon | fstminimize > ../fsts/kati2.binfst
                ! fstprint -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/kati2.binfst > ../fsts/kati2.fst
                with open('../fsts/kati2.fst', 'r') as file:
                # iterate through each line in the file
                    lines = file.readlines()
                    # create a 2D array to store the values
                    data = []
                    # loop through each line and split it into values using spaces
                    for line in lines:
                        values = line.strip().split(" ")
                        # add the row to the 2D array
                        data.append(values)
                    #create final corrected string from fst file
                    array_str=np.array(list(data[0][0]))[6:]
                    shape_temp = np.shape(array_str)[0]
                    array_f=[]
                    for i in range(np.shape(array_str)[0]):
                        if(array_str[i]=='\t'):
                            break
                        else:
                            array_f.append(array_str[i])
                    # array_f=array_str[6:shape_temp-11]
                    corrected_word = "".join(array_f)
                    if(corrected_word==correct_word):
                        score+=1
                    # print("corrected word is: ",corrected_word)
                    corrected_results.append(corrected_word)
                    # print(correct_word," ",wrong_word, " ",corrected_word)
        print()
        print("correct : ","wrong", "->", "corrected ")
        print()
        for i in range(np.shape(corrected_results)[0]):
            print(correct_results[i],":",wrong_results[i],"->",corrected_results[i])
        print()
        print("final accuracy score is: ", (score/num_of_words)*100, "%", " in ", test_set_size, " words")
#test_spell_checker("../data/spell_test.txt", "S", 5)

#### Step 8

In [11]:
! python3 mkfstinput.py abandonned > ../fsts/M.fst
! python3 mkfstinput.py abandoned > ../fsts/N.fst
! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/M.fst > ../fsts/M.binfst
! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/N.fst > ../fsts/N.binfst
! fstarcsort --sort_type=olabel ../fsts/M.binfst ../fsts/M_sorted.binfst
! fstarcsort --sort_type=ilabel ../fsts/N.binfst ../fsts/N_sorted.binfst
! fstcompose ../fsts/M_sorted.binfst ../fsts/L_sorted.binfst ../fsts/ML.binfst
! fstarcsort --sort_type=ilabel ../fsts/ML.binfst ../fsts/ML_sorted.binfst
! fstcompose ../fsts/ML_sorted.binfst ../fsts/N_sorted.binfst ../fsts/MLN.binfst
! fstshortestpath ../fsts/MLN.binfst | fstprint --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms --show_weight_one
! source word_edits.sh abandonned abandoned

12	11	a	a	0
0	0
1	0	<eps>	<eps>	0
2	1	<eps>	<eps>	0
3	2	d	d	0
4	3	e	e	0
5	4	n	n	0
6	5	n	<eps>	1
7	6	o	o	0
8	7	d	d	0
9	8	n	n	0
10	9	a	a	0
11	10	b	b	0
n	<eps>


In [12]:
with open('../data/wiki.txt') as f:
    lines = [line.rstrip().split('\t') for line in f]
edit_dict = {}
with open("../vocab/edits.txt", "w+", newline = '') as out_f:
    for line in lines:
        x = !source word_edits.sh {line[0]} {line[1]}
        out_f.write(x[0] + '\n')
        temp = x[0].split("\t")
        try:
            key = (temp[0], temp[1])
        except:
            print(temp)
            print(line)
            continue
        if key in edit_dict:
            edit_dict[key] = edit_dict[key] + 1
        else:
            edit_dict[key] = 1
    


['FATAL: FstCompiler: Symbol "." is not mapped to any integer arc ilabel, symbol table = ../vocab/chars.syms, source = standard input, line = 2']
['aka', 'a.k.a.']
['FATAL: FstCompiler: Symbol "á" is not mapped to any integer arc ilabel, symbol table = ../vocab/chars.syms, source = standard input, line = 7']
['gauarana', 'guaraná']
['FATAL: FstCompiler: Symbol "1" is not mapped to any integer arc ilabel, symbol table = ../vocab/chars.syms, source = standard input, line = 1']
['ninties', '1990s']


In [13]:
def create_E(charset, frequency_dict, filename):
    total_edits = sum(frequency_dict.values())
    with open(filename, "w") as f:
        for c in charset:
            print(format_arc(0, 0, c, c, weight=0), file=f)   # 1: Every character to itself with weight 0 (no edit)
            if (c, EPS) in frequency_dict:
                print(format_arc(0, 0, c, EPS, weight=-np.log10(frequency_dict[(c, EPS)]/total_edits)), file=f) # 2: Every character to EPS with weight 1 (deletion)
            else:
                print(format_arc(0, 0, c, EPS, weight=10000), file=f)
                      
            if (EPS, c) in frequency_dict:
                print(format_arc(0, 0, EPS, c, weight=-np.log10(frequency_dict[(EPS, c)]/total_edits)), file=f) # 3: EPS to every other character with weight 1 (insertion)
            else:
                print(format_arc(0, 0, EPS, c, weight=10000), file=f)
                      
            
            for other_c in charset:                           # 4: Every character to every other character with weight 1
                if c == other_c:
                    continue # If the characters are the same the weight is 0
                if (c, other_c) in frequency_dict:
                    print(format_arc(0, 0, c, other_c, weight=-np.log10(frequency_dict[(c, other_c)]/total_edits)), file=f) # 3: EPS to every other character with weight 1 (insertion)
                else:
                    print(format_arc(0, 0, c, other_c, weight=10000), file=f)
        print(0, file=f)  # Accepting state
               
create_E(CHARS, edit_dict, "../fsts/E.fst")

In [14]:
! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/E.fst > ../fsts/E.binfst
! fstarcsort --sort_type=olabel ../fsts/E.binfst ../fsts/E_sorted.binfst
! fstcompose ../fsts/E_sorted.binfst ../fsts/V_sorted.binfst ../fsts/EV.binfst

test_spell_checker("../data/spell_test.txt", "EV",10)


correct :  wrong -> corrected 

contented : contenpted -> contented
contented : contende -> contend
contented : contended -> contended
contented : contentid -> contented
beginning : begining -> beginning
problem : problam -> problem
problem : proble -> problem
problem : promblem -> problem
problem : proplen -> people
driven : dirven -> driven
ecstasy : exstacy -> exactly
ecstasy : ecstacy -> ecstasy
juice : guic -> guise
juice : juce -> juice
juice : jucie -> juice
juice : juise -> juice
juice : juse -> just
locally : localy -> local
compare : compair -> compare
pronunciation : pronounciation -> pronouncing
transportability : transportibility -> respectability

final accuracy score is:  57.14285714285714 %  in  10  words


#### Step 9

In [15]:
def create_W(word_dict, filename, num_words=-1):
    total_words = sum(word_dict.values())
    if num_words == -1:
        with open(filename, "w") as f:
            for word in word_dict:
                print(format_arc(0, 0, word, word, weight=-np.log10(word_dict[word]/total_words)), file=f)   
            print(0, file=f)  # Accepting state
    else:
        current_word = 0
        with open(filename, "w") as f:
            for word  in word_dict:
                if current_word >= num_words:
                    break
                print(format_arc(0, 0, word, word, weight=-np.log10(word_dict[word]/total_words)), file=f)   
                current_word = current_word + 1
            print(0, file=f)  # Accepting state
create_W(out_dict, "../fsts/W_arx.fst")

In [16]:
! fstcompile -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W_arx.fst ../fsts/W_arx.binfst
! fstrmepsilon ../fsts/W_arx.binfst | fstdeterminize | fstminimize > ../fsts/W.binfst
! fstprint -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W.binfst > ../fsts/W.fst
! fstcompile -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W.fst > ../fsts/W.binfst
! fstarcsort --sort_type=olabel ../fsts/W.binfst ../fsts/W_sorted.binfst

! fstarcsort --sort_type=olabel ../fsts/EV.binfst ../fsts/EV_sorted.binfst
! fstarcsort --sort_type=olabel ../fsts/S.binfst ../fsts/S_sorted.binfst
! fstcompose ../fsts/S_sorted.binfst ../fsts/W_sorted.binfst ../fsts/LVW.binfst
! fstcompose ../fsts/EV_sorted.binfst ../fsts/W_sorted.binfst ../fsts/EVW.binfst
test_spell_checker("../data/spell_test.txt", "LVW",5)
test_spell_checker("../data/spell_test.txt", "EVW",5)


correct :  wrong -> corrected 

contented : contenpted -> contented
contented : contende -> contend
contented : contended -> contended
contented : contentid -> contented
beginning : begining -> beginning
problem : problam -> problem
problem : proble -> people
problem : promblem -> problem
problem : proplen -> people
driven : dirven -> given
ecstasy : exstacy -> stay
ecstasy : ecstacy -> ecstasy

final accuracy score is:  50.0 %  in  5  words

correct :  wrong -> corrected 

contented : contenpted -> contented
contented : contende -> contend
contented : contended -> contended
contented : contentid -> contented
beginning : begining -> beginning
problem : problam -> problem
problem : proble -> problem
problem : promblem -> problem
problem : proplen -> people
driven : dirven -> driven
ecstasy : exstacy -> exactly
ecstasy : ecstacy -> ecstasy

final accuracy score is:  66.66666666666666 %  in  5  words


In [17]:
create_V(out_dict, "../fsts/V_arx_small.fst", 3)
create_W(out_dict, "../fsts/W_arx_small.fst", 3)
#Create W_small
! fstcompile -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W_arx_small.fst ../fsts/W_arx_small.binfst
! fstrmepsilon ../fsts/W_arx_small.binfst | fstdeterminize | fstminimize > ../fsts/W_small.binfst
! fstprint -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W_small.binfst > ../fsts/W_small.fst
! fstcompile -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W_small.fst > ../fsts/W_small.binfst
! fstdraw --isymbols=../vocab/words.syms --osymbols=../vocab/words.syms -portrait ../fsts/W_small.binfst | dot -Tpng >../documentation/W_small.png

! fstcompile -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V_arx_small.fst ../fsts/V_arx_small.binfst
! fstrmepsilon ../fsts/V_arx_small.binfst | fstdeterminize | fstminimize > ../fsts/V_small.binfst
! fstprint -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V_small.binfst > ../fsts/V_small.fst
! fstcompile -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V_small.fst > ../fsts/V_small.binfst

! fstarcsort --sort_type=olabel ../fsts/V_small.binfst ../fsts/V_small_sorted.binfst
! fstarcsort --sort_type=olabel ../fsts/W_small.binfst ../fsts/W_small_sorted.binfst

! fstcompose ../fsts/V_small_sorted.binfst ../fsts/W_small_sorted.binfst ../fsts/VW_small.binfst
! fstdraw --isymbols=../vocab/chars.syms --osymbols=../vocab/words.syms -portrait ../fsts/VW_small.binfst | dot -Tpng >../documentation/VW_small.png

#### Step 10

In [18]:
! python3 run_evaluation.py ../fsts/S.binfst

contenpted -> contented: contented                                              
contende -> contended: contented                                                
contended -> contended: contented                                               
contentid -> contented: contented                                               
begining -> beginning: beginning                                                
problam -> problem: problem                                                     
proble -> problem: problem                                                      
promblem -> problem: problem                                                    
proplen -> people: problem                                                      
dirven -> given: driven                                                         
exstacy -> ecstasy: ecstasy                                                     
ecstacy -> ecstasy: ecstasy                                                     
guic -> guil: juice         

dessicate -> delicate: desiccate                                                
dessiccate -> delicate: desiccate                                               
clearical -> clerical: clerical                                                 
spledid -> splendid: splendid                                                   
splended -> splendid: splendid                                                  
splened -> silenced: splendid                                                   
splended -> splendid: splendid                                                  
beetween -> between: between                                                    
completly -> completely: completely                                             
acount -> count: account                                                        
cemetary -> momentary: cemetery                                                 
semetary -> secretary: cemetery                                                 
speaical -> special: special

scarcly -> scarcely: scarcely                                                   
scarecly -> scarcely: scarcely                                                  
scarely -> scarcely: scarcely                                                   
scarsely -> scarcely: scarcely                                                  
questionaire -> questioning: questionnaire                                      
experance -> temperance: experience                                             
experiance -> experience: experience                                            
possable -> possible: possible                                                  
reafreshment -> refreshment: refreshment                                        
refreshmant -> refreshment: refreshment                                         
refresment -> refreshment: refreshment                                          
refressmunt -> refreshment: refreshment                                         
embaras -> embark: embarrass

In [19]:
! python3 run_evaluation.py ../fsts/EV.binfst

contenpted -> contented: contented                                              
contende -> contend: contented                                                  
contended -> contended: contented                                               
contentid -> contented: contented                                               
begining -> beginning: beginning                                                
problam -> problem: problem                                                     
proble -> problem: problem                                                      
promblem -> problem: problem                                                    
proplen -> people: problem                                                      
dirven -> driven: driven                                                        
exstacy -> exactly: ecstasy                                                     
ecstacy -> ecstasy: ecstasy                                                     
guic -> guise: juice        

dessicate -> dedicate: desiccate                                                
dessiccate -> dedicate: desiccate                                               
clearical -> clerical: clerical                                                 
spledid -> splendid: splendid                                                   
splended -> splendid: splendid                                                  
splened -> spend: splendid                                                      
splended -> splendid: splendid                                                  
beetween -> between: between                                                    
completly -> completely: completely                                             
acount -> account: account                                                      
cemetary -> mary: cemetery                                                      
semetary -> secretary: cemetery                                                 
speaical -> special: special

scarcly -> scarcely: scarcely                                                   
scarecly -> scarcely: scarcely                                                  
scarely -> scarcely: scarcely                                                   
scarsely -> scarcely: scarcely                                                  
questionaire -> question: questionnaire                                         
experance -> experience: experience                                             
experiance -> experience: experience                                            
possable -> possible: possible                                                  
reafreshment -> refreshment: refreshment                                        
refreshmant -> refreshment: refreshment                                         
refresment -> refreshment: refreshment                                          
refressmunt -> refreshment: refreshment                                         
embaras -> embark: embarrass

In [20]:
! python3 run_evaluation.py ../fsts/LVW.binfst

contenpted -> contented: contented                                              
contende -> contend: contented                                                  
contended -> contended: contented                                               
contentid -> contented: contented                                               
begining -> beginning: beginning                                                
problam -> problem: problem                                                     
proble -> people: problem                                                       
promblem -> problem: problem                                                    
proplen -> people: problem                                                      
dirven -> given: driven                                                         
exstacy -> stay: ecstasy                                                        
ecstacy -> ecstasy: ecstasy                                                     
guic -> in: juice           

dessicate -> delicate: desiccate                                                
dessiccate -> delicate: desiccate                                               
clearical -> clerical: clerical                                                 
spledid -> splendid: splendid                                                   
splended -> splendid: splendid                                                  
splened -> opened: splendid                                                     
splended -> splendid: splendid                                                  
beetween -> between: between                                                    
completly -> completely: completely                                             
acount -> about: account                                                        
cemetary -> every: cemetery                                                     
semetary -> secretary: cemetery                                                 
speaical -> special: special

scarcly -> scarcely: scarcely                                                   
scarecly -> scarcely: scarcely                                                  
scarely -> scarcely: scarcely                                                   
scarsely -> scarcely: scarcely                                                  
questionaire -> question: questionnaire                                         
experance -> experience: experience                                             
experiance -> experience: experience                                            
possable -> possible: possible                                                  
reafreshment -> refreshment: refreshment                                        
refreshmant -> refreshment: refreshment                                         
refresment -> refreshment: refreshment                                          
refressmunt -> refreshment: refreshment                                         
embaras -> ears: embarrass  

In [21]:
! python3 run_evaluation.py ../fsts/EVW.binfst

contenpted -> contented: contented                                              
contende -> contend: contented                                                  
contended -> contended: contented                                               
contentid -> contented: contented                                               
begining -> beginning: beginning                                                
problam -> problem: problem                                                     
proble -> problem: problem                                                      
promblem -> problem: problem                                                    
proplen -> people: problem                                                      
dirven -> driven: driven                                                        
exstacy -> exactly: ecstasy                                                     
ecstacy -> ecstasy: ecstasy                                                     
guic -> i: juice            

dessicate -> delicate: desiccate                                                
dessiccate -> delicate: desiccate                                               
clearical -> clerical: clerical                                                 
spledid -> splendid: splendid                                                   
splended -> splendid: splendid                                                  
splened -> spend: splendid                                                      
splended -> splendid: splendid                                                  
beetween -> between: between                                                    
completly -> completely: completely                                             
acount -> account: account                                                      
cemetary -> mary: cemetery                                                      
semetary -> secretary: cemetery                                                 
speaical -> special: special

scarcly -> scarcely: scarcely                                                   
scarecly -> scarcely: scarcely                                                  
scarely -> scarcely: scarcely                                                   
scarsely -> scarcely: scarcely                                                  
questionaire -> question: questionnaire                                         
experance -> experience: experience                                             
experiance -> experience: experience                                            
possable -> possible: possible                                                  
reafreshment -> refreshment: refreshment                                        
refreshmant -> refreshment: refreshment                                         
refresment -> refreshment: refreshment                                          
refressmunt -> refreshment: refreshment                                         
embaras -> bars: embarrass  

#### Step 11

In [22]:
def create_E_add_one(charset, frequency_dict, filename):
    total_edits = sum(frequency_dict.values())
    V = 2*len(charset) + len(charset)*(len(charset)-1)
    with open(filename, "w") as f:
        for c in charset:
            print(format_arc(0, 0, c, c, weight=0), file=f)   # 1: Every character to itself with weight 0 (no edit)
            if (c, EPS) in frequency_dict:
                print(format_arc(0, 0, c, EPS, weight=-np.log10((frequency_dict[(c, EPS)]+1)/(total_edits+V))), file=f) # 2: Every character to EPS with weight 1 (deletion)
            else:
                print(format_arc(0, 0, c, EPS, weight=-np.log10((1)/(total_edits+V))), file=f)
                      
            if (EPS, c) in frequency_dict:
                print(format_arc(0, 0, EPS, c, weight=-np.log10((frequency_dict[(EPS, c)]+1)/(total_edits+V))), file=f) # 3: EPS to every other character with weight 1 (insertion)
            else:
                print(format_arc(0, 0, EPS, c, weight=-np.log10((1)/(total_edits+V))), file=f)
                      
            
            for other_c in charset:                           # 4: Every character to every other character with weight 1
                if c == other_c:
                    continue # If the characters are the same the weight is 0
                if (c, other_c) in frequency_dict:
                    print(format_arc(0, 0, c, other_c, weight=-np.log10((frequency_dict[(c, other_c)]+1)/(total_edits+V))), file=f) # 3: EPS to every other character with weight 1 (insertion)
                else:
                    print(format_arc(0, 0, c, other_c, weight=-np.log10((1)/(total_edits+V))), file=f)
        print(0, file=f)  # Accepting state
               
create_E_add_one(CHARS, edit_dict, "../fsts/E_add_one.fst")

In [23]:
! fstcompile --isymbols=../vocab/chars.syms --osymbols=../vocab/chars.syms ../fsts/E_add_one.fst > ../fsts/E_add_one.binfst
! fstarcsort --sort_type=olabel ../fsts/E_add_one.binfst ../fsts/E_add_one_sorted.binfst
! fstcompose ../fsts/E_add_one_sorted.binfst ../fsts/V_sorted.binfst ../fsts/E_AO_V.binfst

test_spell_checker("../data/spell_test.txt", "E_AO_V",10)


correct :  wrong -> corrected 

contented : contenpted -> contented
contented : contende -> contend
contented : contended -> contended
contented : contentid -> contented
beginning : begining -> beginning
problem : problam -> problem
problem : proble -> problem
problem : promblem -> problem
problem : proplen -> people
driven : dirven -> driven
ecstasy : exstacy -> ecstasy
ecstasy : ecstacy -> ecstasy
juice : guic -> guil
juice : juce -> juice
juice : jucie -> juice
juice : juise -> juice
juice : juse -> just
locally : localy -> local
compare : compair -> compare
pronunciation : pronounciation -> provocation
transportability : transportibility -> respectability

final accuracy score is:  61.904761904761905 %  in  10  words


In [24]:
! python3 run_evaluation.py ../fsts/E_AO_V.binfst

contenpted -> contented: contented                                              
contende -> contend: contented                                                  
contended -> contended: contented                                               
contentid -> contented: contented                                               
begining -> beginning: beginning                                                
problam -> problem: problem                                                     
proble -> problem: problem                                                      
promblem -> problem: problem                                                    
proplen -> people: problem                                                      
dirven -> driven: driven                                                        
exstacy -> ecstasy: ecstasy                                                     
ecstacy -> ecstasy: ecstasy                                                     
guic -> guil: juice         

dessicate -> dedicate: desiccate                                                
dessiccate -> dedicate: desiccate                                               
clearical -> clerical: clerical                                                 
spledid -> splendid: splendid                                                   
splended -> splendid: splendid                                                  
splened -> spend: splendid                                                      
splended -> splendid: splendid                                                  
beetween -> between: between                                                    
completly -> completely: completely                                             
acount -> account: account                                                      
cemetary -> century: cemetery                                                   
semetary -> secretary: cemetery                                                 
speaical -> special: special

scarcly -> scarcely: scarcely                                                   
scarecly -> scarcely: scarcely                                                  
scarely -> scarcely: scarcely                                                   
scarsely -> scarcely: scarcely                                                  
questionaire -> question: questionnaire                                         
experance -> experience: experience                                             
experiance -> experience: experience                                            
possable -> possible: possible                                                  
reafreshment -> refreshment: refreshment                                        
refreshmant -> refreshment: refreshment                                         
refresment -> refreshment: refreshment                                          
refressmunt -> refreshment: refreshment                                         
embaras -> embark: embarrass

In [25]:
en_50 = "../data/en_50k.txt"
en_50_words = ""
with open(en_50, 'r') as file:
    en_50_txt = file.read()
en_50_txt = en_50_txt.split('\n')
for wordCount in en_50_txt[:-1]:
    word, count = wordCount.split(' ')
    en_50_words = en_50_words + word + " "
    if word in out_dict:
        out_dict[word] = out_dict[word] + int(count)
    else:
        out_dict[word] = int(count)
create_W(out_dict, "../fsts/W_2_arx.fst")
create_V(out_dict, "../fsts/V_2_arx.fst")
write_syms(gutenberg_txt + " " + 5*en_50_words)

In [26]:
! fstcompile -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V_2_arx.fst ../fsts/V_2_arx.binfst
! fstrmepsilon ../fsts/V_2_arx.binfst ../fsts/V_2_arx.binfst
! fstdeterminize ../fsts/V_2_arx.binfst ../fsts/V_2_arx.binfst
! fstminimize ../fsts/V_2_arx.binfst ../fsts/V_2.binfst
! fstprint -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V_2.binfst > ../fsts/V_2.fst
! fstcompile -isymbols=../vocab/chars.syms -osymbols=../vocab/words.syms ../fsts/V_2.fst > ../fsts/V_2.binfst
! fstarcsort --sort_type=olabel ../fsts/V_2.binfst ../fsts/V_2_sorted.binfst

! fstcompile -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W_2_arx.fst ../fsts/W_2_arx.binfst
! fstrmepsilon ../fsts/W_2_arx.binfst | fstdeterminize | fstminimize > ../fsts/W_2.binfst
! fstprint -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W_2.binfst > ../fsts/W_2.fst
! fstcompile -isymbols=../vocab/words.syms -osymbols=../vocab/words.syms ../fsts/W_2.fst > ../fsts/W_2.binfst
! fstarcsort --sort_type=olabel ../fsts/W_2.binfst ../fsts/W_2_sorted.binfst

! fstarcsort --sort_type=olabel ../fsts/E.binfst ../fsts/E_sorted.binfst
! fstarcsort --sort_type=olabel ../fsts/S.binfst ../fsts/S_sorted.binfst

! fstcompose ../fsts/L_sorted.binfst ../fsts/V_2_sorted.binfst ../fsts/LV_2.binfst
! fstcompose ../fsts/E_sorted.binfst ../fsts/V_2_sorted.binfst ../fsts/EV_2.binfst

! fstarcsort --sort_type=olabel ../fsts/LV_2.binfst ../fsts/LV_2.binfst
! fstarcsort --sort_type=olabel ../fsts/EV_2.binfst ../fsts/EV_2.binfst

! fstcompose ../fsts/LV_2.binfst ../fsts/W_2_sorted.binfst ../fsts/LVW_2.binfst
! fstcompose ../fsts/EV_2.binfst ../fsts/W_2_sorted.binfst ../fsts/EVW_2.binfst
test_spell_checker("../data/spell_test.txt", "LVW_2",10)
test_spell_checker("../data/spell_test.txt", "EVW_2",10)


correct :  wrong -> corrected 

contented : contenpted -> contented
contented : contende -> continue
contented : contended -> intended
contented : contentid -> content
beginning : begining -> beginning
problem : problam -> problem
problem : proble -> problem
problem : promblem -> problem
problem : proplen -> people
driven : dirven -> didn
ecstasy : exstacy -> stay
ecstasy : ecstacy -> stay
juice : guic -> i
juice : juce -> just
juice : jucie -> just
juice : juise -> just
juice : juse -> just
locally : localy -> local
compare : compair -> company
pronunciation : pronounciation -> pronunciation
transportability : transportibility -> responsibility

final accuracy score is:  28.57142857142857 %  in  10  words

correct :  wrong -> corrected 

contented : contenpted -> contented
contented : contende -> contender
contented : contended -> contended
contented : contentid -> contented
beginning : begining -> beginning
problem : problam -> problem
problem : proble -> problem
problem : promblem 

In [27]:
! python3 run_evaluation.py ../fsts/LVW_2.binfst

contenpted -> contented: contented                                              
contende -> continue: contented                                                 
contended -> intended: contented                                                
contentid -> content: contented                                                 
begining -> beginning: beginning                                                
problam -> problem: problem                                                     
proble -> problem: problem                                                      
promblem -> problem: problem                                                    
proplen -> people: problem                                                      
dirven -> didn: driven                                                          
exstacy -> stay: ecstasy                                                        
ecstacy -> stay: ecstasy                                                        
guic -> i: juice            

dessicate -> delicate: desiccate                                                
dessiccate -> delicate: desiccate                                               
clearical -> clerical: clerical                                                 
spledid -> splendid: splendid                                                   
splended -> splendid: splendid                                                  
splened -> spend: splendid                                                      
splended -> splendid: splendid                                                  
beetween -> between: between                                                    
completly -> completely: completely                                             
acount -> about: account                                                        
cemetary -> cemetery: cemetery                                                  
semetary -> secretary: cemetery                                                 
speaical -> special: special

scarcly -> scary: scarcely                                                      
scarecly -> scared: scarcely                                                    
scarely -> scared: scarcely                                                     
scarsely -> scarcely: scarcely                                                  
questionaire -> questionnaire: questionnaire                                    
experance -> experience: experience                                             
experiance -> experience: experience                                            
possable -> possible: possible                                                  
reafreshment -> refreshment: refreshment                                        
refreshmant -> refreshment: refreshment                                         
refresment -> represent: refreshment                                            
refressmunt -> refreshment: refreshment                                         
embaras -> embarrass: embarr

In [28]:
! python3 run_evaluation.py ../fsts/EVW_2.binfst

contenpted -> contented: contented                                              
contende -> contender: contented                                                
contended -> contended: contented                                               
contentid -> contented: contented                                               
begining -> beginning: beginning                                                
problam -> problem: problem                                                     
proble -> problem: problem                                                      
promblem -> problem: problem                                                    
proplen -> people: problem                                                      
dirven -> driven: driven                                                        
exstacy -> exactly: ecstasy                                                     
ecstacy -> ecstasy: ecstasy                                                     
guic -> guy: juice          

dessicate -> delicate: desiccate                                                
dessiccate -> desiccated: desiccate                                             
clearical -> clerical: clerical                                                 
spledid -> splendid: splendid                                                   
splended -> splendid: splendid                                                  
splened -> spend: splendid                                                      
splended -> splendid: splendid                                                  
beetween -> between: between                                                    
completly -> completely: completely                                             
acount -> account: account                                                      
cemetary -> cemetery: cemetery                                                  
semetary -> cemetery: cemetery                                                  
speaical -> special: special

scarcly -> scarcely: scarcely                                                   
scarecly -> scarcely: scarcely                                                  
scarely -> scarcely: scarcely                                                   
scarsely -> scarcely: scarcely                                                  
questionaire -> questionnaire: questionnaire                                    
experance -> experience: experience                                             
experiance -> experience: experience                                            
possable -> possible: possible                                                  
reafreshment -> refreshment: refreshment                                        
refreshmant -> refreshment: refreshment                                         
refresment -> refreshment: refreshment                                          
refressmunt -> refreshment: refreshment                                         
embaras -> embarrass: embarr

#### Step 12

In [127]:
nltk.download("punkt")
!python3 fetch_gutenberg_upper.py > ../data/gutenberg2.txt
# gut_path = "../data/gutenberg.txt"
# with open('../data/gutenberg.txt', 'r') as file:
#     gutenberg_txt = file.read()
# tokenized_sentences = sent_tokenize(gutenberg_txt)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/subdiduntil2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/subdiduntil2/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [6]:
f = open("../data/gutenberg2.txt")
gut_txt = f.read()
tokenized_sentences = sent_tokenize(gut_txt)

In [7]:
def clean_text(s): #initial clean function (with conversion to lower)
    s = s.strip()  # Strip leading / trailing spaces
    s = s.lower()  # Convert to lowercase
    s = contractions.fix(s)  # e.g. don't -> do not, you're -> you are
    s = re.sub("\s+", " ", s)  # Strip multiple whitespace
    s = re.sub(r"[^a-z\s]", " ", s)  # Keep only lowercase letters and spaces
    return s

cleaned_sentences = [clean_text(sentence) for sentence in tokenized_sentences]

words = [word_tokenize(sentence) for sentence in cleaned_sentences] #tokenize every word in each sentence
preprocessed=words

In [135]:
# Initialize word2vec. Context is taken as the 2 previous and 2 next words (needs to be further trained)
# model_10 =Word2Vec(preprocessed, size=100, window=5,min_count=1,workers=4)
# # Train the model for 10 epochs
# model_10.train(preprocessed, total_examples=len(preprocessed), epochs=10) 
# model_10.save("../models/w2v10.model")
# print("model 10 finished")


# model_100 =Word2Vec(preprocessed, size=100, window=5, min_count=1, workers=4)
# # Train the model for 10 epochs
# model_100.train(preprocessed, total_examples=len(preprocessed), epochs=100)
# model_100.save("../models/w2v100.model")
# print("model 100 finished")

model 10 finished
model 100 finished


In [53]:
#test for different models 
def test1(model): 
    voc = model.wv.index2word
    words_to_check = ["bible","book","bank","water"]
    for word in words_to_check:
        sim = model.wv.most_similar(word,topn=5)
        print(word," -> ",sim)
        
print("model_10 results")
modelt10=Word2Vec.load("../models/w2v10.model")
test1(modelt10)
print()

print("model_100 results")
modelt100=Word2Vec.load("../models/w2v100.model")
test1(modelt100)
print()


model_10 results
bible  ->  [('title', 0.4902319312095642), ('republic', 0.4884238541126251), ('gentry', 0.48822328448295593), ('grog', 0.48778092861175537), ('hong', 0.4873921275138855)]
book  ->  [('written', 0.6561650037765503), ('chronicles', 0.6195775270462036), ('volume', 0.5716540217399597), ('letter', 0.5286608934402466), ('note', 0.5094970464706421)]
bank  ->  [('beach', 0.6997634172439575), ('floor', 0.691679835319519), ('wharf', 0.6782059073448181), ('terrace', 0.6756478548049927), ('ridge', 0.673412024974823)]
water  ->  [('waters', 0.6341620683670044), ('bushes', 0.5385381579399109), ('springs', 0.5329419374465942), ('brook', 0.5273190140724182), ('rivers', 0.5260152816772461)]

model_100 results
bible  ->  [('arrum', 0.3908499777317047), ('exodus', 0.38544797897338867), ('untaken', 0.38398146629333496), ('hieroglyphics', 0.37933698296546936), ('preacher', 0.37702327966690063)]
book  ->  [('chronicles', 0.5780780911445618), ('written', 0.5374575853347778), ('comedy', 0.482

In [54]:
# final pipeline for 1000 epoch model
# model_1000 =Word2Vec(preprocessed, size=100, window=5, min_count=1, workers=4)
# # Train the model for 1000 epochs
# model_1000.train(preprocessed, total_examples=len(preprocessed), epochs=1000)
# model_1000.save("../models/w2v1000.model")
print("model_1000 results")

modelt1000=Word2Vec.load("../models/w2v1000.model")
test1(modelt1000)

model_1000 results
bible  ->  [('davenport', 0.39520949125289917), ('proprietary', 0.3803219795227051), ('buckled', 0.3767137825489044), ('scarf', 0.3730965256690979), ('book', 0.3710101246833801)]
book  ->  [('chronicles', 0.5138381719589233), ('written', 0.503126859664917), ('raze', 0.4715029001235962), ('temple', 0.45585402846336365), ('scedule', 0.45351463556289673)]
bank  ->  [('pool', 0.5486129522323608), ('wall', 0.5361111164093018), ('table', 0.49621516466140747), ('foot', 0.461711585521698), ('side', 0.4513593316078186)]
water  ->  [('waters', 0.6290057301521301), ('river', 0.5684349536895752), ('sea', 0.5539792776107788), ('liquid', 0.5368390679359436), ('streams', 0.5314413905143738)]


In [55]:
triplets = [["girls","queen","kings"], ["good","tall","taller"], ["france","paris","london"]]

def test2(model,triplets): 
    voc = model.wv.index2word
    i=0
    for w1, w2, w3 in triplets:
        sim = model.wv.most_similar(positive=[w1, w3],negative=[w2],topn=3)
        print(triplets[i]," -> ", sim)
        i+=1
print("model_10 results")
test2(modelt10,triplets)
print()
print("model_100 results")
test2(modelt100,triplets)
print()
print("model_1000 results")
test2(modelt1000,triplets)
print()


model_10 results
['girls', 'queen', 'kings']  ->  [('men', 0.5662633776664734), ('peasants', 0.5399688482284546), ('creatures', 0.5313206315040588)]
['good', 'tall', 'taller']  ->  [('better', 0.4536677896976471), ('worse', 0.4290015697479248), ('wiser', 0.42885762453079224)]
['france', 'paris', 'london']  ->  [('england', 0.533356785774231), ('highbury', 0.5147027969360352), ('parish', 0.47496679425239563)]

model_100 results
['girls', 'queen', 'kings']  ->  [('men', 0.5164756774902344), ('moralists', 0.4826558232307434), ('dwellings', 0.4695836901664734)]
['good', 'tall', 'taller']  ->  [('better', 0.5012955665588379), ('comfort', 0.39991235733032227), ('fool', 0.39102932810783386)]
['france', 'paris', 'london']  ->  [('sussex', 0.4982496500015259), ('england', 0.44794729351997375), ('inn', 0.4472661018371582)]

model_1000 results
['girls', 'queen', 'kings']  ->  [('parts', 0.49015381932258606), ('men', 0.46490103006362915), ('cities', 0.4524120092391968)]
['good', 'tall', 'taller'] 

In [69]:
model3 = KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True, limit=1000000)
model3.save("../models/word2vec_google.model")

In [56]:
model_google=KeyedVectors.load("../models/word2vec_google.model")
print()
print("model_google results for test1")
test1(model_google)
print()
print("model_google results for test2")
test2(model_google,triplets)


model_google results for test1


/tmp/ipykernel_45027/4239279239.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  voc = model.wv.index2word
/tmp/ipykernel_45027/4239279239.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  sim = model.wv.most_similar(word,topn=5)


bible  ->  [('Bible', 0.7367781400680542), ('bibles', 0.6052597761154175), ('Holy_Bible', 0.5989601612091064), ('scriptures', 0.574568510055542), ('scripture', 0.5697901248931885)]
book  ->  [('tome', 0.7485830783843994), ('books', 0.7379178404808044), ('memoir', 0.7302927374839783), ('paperback_edition', 0.6868364810943604), ('autobiography', 0.6741527915000916)]
bank  ->  [('banks', 0.7440759539604187), ('banking', 0.690161406993866), ('Bank', 0.6698698997497559), ('lender', 0.6342284679412842), ('banker', 0.6092953681945801)]
water  ->  [('potable_water', 0.6799106597900391), ('Water', 0.6706870794296265), ('sewage', 0.6619377136230469), ('groundwater', 0.6588345766067505), ('Floridan_aquifer', 0.6422533988952637)]

model_google results for test2
['girls', 'queen', 'kings']  ->  [('boys', 0.7183727025985718), ('men', 0.5000519752502441), ('teenagers', 0.49677687883377075)]
['good', 'tall', 'taller']  ->  [('better', 0.7245721220970154), ('nicer', 0.5474838018417358), ('great', 0.543

/tmp/ipykernel_45027/3547651740.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  voc = model.wv.index2word
/tmp/ipykernel_45027/3547651740.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  sim = model.wv.most_similar(positive=[w1, w3],negative=[w2],topn=3)


#### Step 13

In [14]:
model=Word2Vec.load("../models/w2v100.model")
with open("../data/embeddings.tsv", "w") as f:
    words = model.wv.vocab.keys()
    # print(words)
    for word in words:
        vector = list(model.wv.get_vector(word))
        # print(vector)
        for vector_element in vector:
            # print(vector_element)
            print(vector_element, file=f, end="\t")
        print(file=f)

with open("../data/metadata.tsv", "w") as f:
    words = model.wv.vocab.keys()
    for word in words:
        print(word, file=f)

#### Step14

In [25]:
!python3 w2v_sentiment_analysis.py

loaded models
read stuff
w2v_sentiment_analysis.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  corpus = np.array(pos + neg)
create nbows_1
w2v_sentiment_analysis.py:85: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  init_vec = np.add(init_vec, model[word])
training of LR started
Info for model1
The accuracy of the model is  0.7475
create nbows_2
w2v_sentiment_analysis.py:81: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model.wv.index2word)
training of LR started
Info for model2
The accuracy of the model is  0.8347
